In [107]:
import langchain_community 
import langchain
import langchain_openai

In [108]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '<api-key>'

In [110]:
os.environ['OPENAI_API_KEY'] = '<api-key>'

In [111]:
import json

# Load the JSON data
with open("../../monsters_response.json", "r") as file:
    json_data = json.load(file)

# Extract the monsters list
monsters = json_data["monsters"]

print(monsters)

[{'monster_id': 'deathclaw_1', 'lon': 85.5487, 'lat': 17.9072}, {'monster_id': 'deathclaw_2', 'lon': 87.3032, 'lat': 19.4866}, {'monster_id': 'super_mutant_1', 'lon': 83.1134, 'lat': 19.163}, {'monster_id': 'super_mutant_2', 'lon': 82.8223, 'lat': 18.762}, {'monster_id': 'super_mutant_3', 'lon': 87.0667, 'lat': 20.6589}, {'monster_id': 'super_mutant_4', 'lon': 83.4131, 'lat': 22.1663}, {'monster_id': 'super_mutant_5', 'lon': 81.6439, 'lat': 18.7261}, {'monster_id': 'radscorpion_1', 'lon': 87.2045, 'lat': 18.7888}, {'monster_id': 'radscorpion_2', 'lon': 86.0467, 'lat': 21.2248}, {'monster_id': 'radscorpion_3', 'lon': 86.5529, 'lat': 21.1394}, {'monster_id': 'radscorpion_4', 'lon': 83.6146, 'lat': 20.4416}, {'monster_id': 'yao_guai_1', 'lon': 85.4507, 'lat': 18.2524}, {'monster_id': 'yao_guai_2', 'lon': 83.6133, 'lat': 21.7572}, {'monster_id': 'yao_guai_3', 'lon': 86.3814, 'lat': 20.8939}, {'monster_id': 'feral_ghoul_1', 'lon': 85.8137, 'lat': 21.4151}, {'monster_id': 'feral_ghoul_2', 'l

In [112]:
from langchain_core.documents import Document

# Convert monsters to LangChain Document objects
docs = [
    Document(
        page_content=f"Monster ID: {monster['monster_id']}, Location: ({monster['lat']}, {monster['lon']})",
        metadata={"monster_id": monster["monster_id"], "latitude": monster["lat"], "longitude": monster["lon"]}
    )
    for monster in monsters
]

# Print the documents
for doc in docs:
    print(doc)

page_content='Monster ID: deathclaw_1, Location: (17.9072, 85.5487)' metadata={'monster_id': 'deathclaw_1', 'latitude': 17.9072, 'longitude': 85.5487}
page_content='Monster ID: deathclaw_2, Location: (19.4866, 87.3032)' metadata={'monster_id': 'deathclaw_2', 'latitude': 19.4866, 'longitude': 87.3032}
page_content='Monster ID: super_mutant_1, Location: (19.163, 83.1134)' metadata={'monster_id': 'super_mutant_1', 'latitude': 19.163, 'longitude': 83.1134}
page_content='Monster ID: super_mutant_2, Location: (18.762, 82.8223)' metadata={'monster_id': 'super_mutant_2', 'latitude': 18.762, 'longitude': 82.8223}
page_content='Monster ID: super_mutant_3, Location: (20.6589, 87.0667)' metadata={'monster_id': 'super_mutant_3', 'latitude': 20.6589, 'longitude': 87.0667}
page_content='Monster ID: super_mutant_4, Location: (22.1663, 83.4131)' metadata={'monster_id': 'super_mutant_4', 'latitude': 22.1663, 'longitude': 83.4131}
page_content='Monster ID: super_mutant_5, Location: (18.7261, 81.6439)' me

In [113]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)

# Split the documents
split_docs = text_splitter.split_documents(docs)

print(split_docs)

[Document(metadata={'monster_id': 'deathclaw_1', 'latitude': 17.9072, 'longitude': 85.5487}, page_content='Monster ID: deathclaw_1, Location: (17.9072, 85.5487)'), Document(metadata={'monster_id': 'deathclaw_2', 'latitude': 19.4866, 'longitude': 87.3032}, page_content='Monster ID: deathclaw_2, Location: (19.4866, 87.3032)'), Document(metadata={'monster_id': 'super_mutant_1', 'latitude': 19.163, 'longitude': 83.1134}, page_content='Monster ID: super_mutant_1, Location: (19.163, 83.1134)'), Document(metadata={'monster_id': 'super_mutant_2', 'latitude': 18.762, 'longitude': 82.8223}, page_content='Monster ID: super_mutant_2, Location: (18.762, 82.8223)'), Document(metadata={'monster_id': 'super_mutant_3', 'latitude': 20.6589, 'longitude': 87.0667}, page_content='Monster ID: super_mutant_3, Location: (20.6589, 87.0667)'), Document(metadata={'monster_id': 'super_mutant_4', 'latitude': 22.1663, 'longitude': 83.4131}, page_content='Monster ID: super_mutant_4, Location: (22.1663, 83.4131)'), D

In [127]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings

# Initialize embeddings (ensure your API key is set up for OpenAI)
embeddings = OpenAIEmbeddings()

# Store documents in FAISS
vector_store = FAISS.from_documents(split_docs, embeddings)

# Save the vector store locally
vector_store.save_local("faiss_index")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# Load the retriever
retriever = vector_store.as_retriever()

# Initialize the language model
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Set up the RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

# Query the system
query = "Where is deathclaw_1 located?"
response = rag_chain.run(query)

print(response)